## Ejercicio: *Time Series Forecast*
Para este ejercicio vamos a predecir cuál será la demanda de pasajeros de una aerolinea, para poder anticiparse a las contrataciones de personal, mantenimiento de las aeronaves y gestión de inventario y comidas.

Para ello, se pide:
1. Carga los datos (AirPassengers.csv) y representa la serie. ¿Hay estacionalidad? ¿Cada cuanto?
2. Crea en una gráfica la variable original + su media obtenida mediante una *rolling window* con el valor de *estacionalidad* obtenido en el apartado anterior. Tienes que usar la función ``rolling()`` del DataFrame.
3. Comprueba de manera estadística si la serie es o no estacionaria.
4. Aplica una transformación logarítmica sobre los datos para mejorar el proceso de transformación de tu series temporal a estacionaria. Acuérdate después del *forecast* de invertir la transformación.
5. Divide en *train* y *test*. Guarda 20 muestras para *test*.
6. Crea tu primer modelo ARIMA. Habrá varias combinaciones en función de sus hiperparámetros... Mide el MAE y RMSE del modelo en predicción. Ten en cuenta el parámetro "m" de la función ARIMA, mediante el cual se establece la estacionalidad.
7. Representa en una gráfica los datos de *test* y tus predicciones.
8. Prueba un *decission tree* y un *random forest*, a ver qué *performance* presentan.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Carga datos y representa la serie

In [ ]:
df = pd.read_csv("data/AirPassengers.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.info()

In [ ]:
pd.date_range(df['date'].min(), df['date'].max(), freq="MS")

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(df['value']);

## 2. Crea en una gráfica la variable original + su media obtenida mediante una *rolling window* comparando el valor de la estacionalidad obtenido en el apartado anterior

In [ ]:
df['MA']= df['value'].rolling(5).mean()
df.head(10)

In [ ]:
plt.figure(figsize=(15,10))
# plt.plot(df['value'], c='b');
plt.plot(df['MA'], c='r');

## 3. Comprueba de manera estadística si la serie es o no estacionaria.

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df['value'].values)
result[1]


## 4. Aplica una transformación logarítmica

In [ ]:
df['value']

In [ ]:
np.log(df['value'])

In [ ]:
df['value'] = np.log(df['value'])
df

In [ ]:
df.set_index('date', inplace=True, drop=True)

In [ ]:
# df.drop(columns=['date', 'MA'], inplace=True)
df

## 5. Divide en *train* y *test*. Guarda 20 muestras para *test*.

In [ ]:
144-20

In [ ]:
train = df.iloc[:124,]
test = df.iloc[124:,]

In [ ]:
print(train.shape)
print(test.shape)

## 6. Crea tu primer modelo ARIMA

In [ ]:
from pmdarima.arima import auto_arima, ARIMA

In [ ]:
train_array = np.array(train['value'].astype(float))
train_array

In [ ]:
model = auto_arima(train_array,
                   start_p = 1,
                   start_q = 1,
                   max_p = 5,
                   max_q = 5,
                   max_d = 3,
                   trace=True)

In [ ]:
y_pred = model.predict(20)

In [ ]:
display(y_pred, test.values)

## 7. Representa en una gráfica los datos de *test* y tus predicciones.

In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(test.value, y_pred))

In [ ]:
plt.plot(test.value.values);
plt.plot(y_pred, c='r');

In [ ]:
model = auto_arima(train_array,
                   start_p = 1,
                   start_q = 1,
                   max_p = 5,
                   max_q = 5,
                   max_d = 3,
                   m=12,
                   stationary=False,
                   trace=True)

In [ ]:
y_pred = model.predict(20)
print(mean_absolute_error(test.value, y_pred))
plt.plot(test.value.values);
plt.plot(y_pred, c='r');

In [ ]:
# y_pred = model.predict(20)
# print(mean_absolute_error(test.values, y_pred))
plt.plot(test.value.values);
plt.plot(model.predict(40), c='r');

In [ ]:
np.exp(test.value.values)

In [ ]:
np.exp(y_pred)

In [ ]:
plt.plot(np.exp(test.value.values));
plt.plot(np.exp(y_pred), c='r');

## 8. Prueba otros modelos, a ver qué *performance* presentan.

In [ ]:
for i in range(12,0,-1):
    df['t-'+str(i)] = df['value'].shift(i)

df.dropna(inplace=True)
df

In [ ]:
len(df)-20

In [ ]:
X = df.drop(columns=['value', 'MA'])
y = df['value']

In [ ]:
X

In [ ]:
X_train = X.iloc[:112,]
X_test = X.iloc[112:,]
y_train = y[:112]
y_test  = y[112:]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y_test.values

In [ ]:
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(max_depth=5)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(mean_absolute_error(y_test.values,y_pred))

plt.plot(y_test.values)
plt.plot(y_pred, c='r')